[View in Colaboratory](https://colab.research.google.com/github/gowtham91m/cats-and-dogs-classification/blob/master/pytorch_cats_and_dogs_classificatoin.ipynb)

In [0]:
%%capture
!pip install kaggle
!pip install torch
!pip install torchvision

In [0]:
from zipfile import ZipFile
import io, cv2, fnmatch, shutil, os, getpass, subprocess, random
import numpy as np
from time import time
from glob import glob

In [3]:
os.chdir('/content')
if 'kaggle.txt' not in os.listdir('/content'):
  from google.colab import files
  downloaded = files.upload()

Saving kaggle.txt to kaggle.txt


In [5]:
os.chdir('/content')
with open('kaggle.txt') as f: key = f.read()
os.environ['KAGGLE_USERNAME'], os.environ['KAGGLE_KEY'] = "gowham91m", key
if 'cats_dogs' in os.listdir('/content'):shutil.rmtree('/content/cats_dogs')
os.mkdir('/content/cats_dogs')
os.chdir('/content/cats_dogs')
!kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 36.8MB/s]
 95% 258M/271M [00:01<00:00, 154MB/s]
100% 271M/271M [00:01<00:00, 148MB/s]
 97% 529M/543M [00:03<00:00, 163MB/s]
100% 543M/543M [00:03<00:00, 149MB/s]


In [6]:
!unzip -q -o train.zip
!unzip -q -o test1.zip

cat_pattern = '*cat.*.jpg'
dog_pattern = '*dog.*.jpg'

images = glob('/content/cats_dogs/train/*.jpg', recursive=True)
cats = fnmatch.filter(images,cat_pattern)
dogs = fnmatch.filter(images,dog_pattern)

os.listdir('/content/cats_dogs')
if 'data' not in os.listdir('/content/cats_dogs'):os.mkdir('/content/cats_dogs/data')
if 'train' not in os.listdir('/content/cats_dogs/data'):os.mkdir('/content/cats_dogs/data/train')
if 'dogs' not in os.listdir('/content/cats_dogs/data/train'):os.mkdir('/content/cats_dogs/data/train/dogs')
if 'cats' not in os.listdir('/content/cats_dogs/data/train'):os.mkdir('/content/cats_dogs/data/train/cats')
  
if 'val' not in os.listdir('/content/cats_dogs/data'):os.mkdir('/content/cats_dogs/data/val')
if 'dogs' not in os.listdir('/content/cats_dogs/data/val'):os.mkdir('/content/cats_dogs/data/val/dogs')
if 'cats' not in os.listdir('/content/cats_dogs/data/val'):os.mkdir('/content/cats_dogs/data/val/cats')

train_dogs_path = '/content/cats_dogs/data/train/dogs'
train_cats_path = '/content/cats_dogs/data/train/cats'

val_dogs_path = '/content/cats_dogs/data/val/dogs'
val_cats_path = '/content/cats_dogs/data/val/cats'

for file in cats: shutil.copy2(file, train_cats_path)
for file in dogs: shutil.copy2(file, train_dogs_path)
  
  
# split train date into train and validation
train_len = len(os.listdir('/content/cats_dogs/data/train/dogs'))
val_len = train_len * 0.3
val_dogs = random.sample(os.listdir(train_dogs_path),int(val_len))
val_cats = random.sample(os.listdir(train_cats_path),int(val_len))


for file in val_dogs:
  try: shutil.move(os.path.join(train_dogs_path,file), val_dogs_path)
  except: pass
for file in val_cats:
  try: shutil.move(os.path.join(train_cats_path,file), val_cats_path)
  except: pass
  
  
  
print(len(os.listdir(train_cats_path)))
print(len(os.listdir(val_cats_path)))

print(len(os.listdir(train_dogs_path)))
print(len(os.listdir(val_dogs_path)))

print('total train samples ', len(os.listdir(train_cats_path)) + len(os.listdir(train_dogs_path)))
print('total train samples ', len(os.listdir(val_cats_path)) + len(os.listdir(val_dogs_path)))

8750
3750
8750
3750
total train samples  17500
total train samples  7500


##Train on pytorch

In [0]:
%%capture
!pip install Pillow==4.0.0

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from tqdm import tqdm
from torch.autograd import Variable

In [0]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transformer = transforms.Compose([
                                        transforms.Resize((150,150)),           
                                        transforms.RandomHorizontalFlip(),  
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=mean,std=std)])  


val_transformer = transforms.Compose([
                                        transforms.Resize((150,150)),       
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=mean,std=std)])  

train_data = ImageFolder('/content/cats_dogs/data/train', transform = train_transformer)
val_data = ImageFolder('/content/cats_dogs/data/val', transform = val_transformer)

train_loader = DataLoader(train_data,
                        batch_size=128,
                        shuffle=True)

val_loader = DataLoader(val_data,
                        batch_size=128,
                        shuffle=False)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3)
        self.conv2 = nn.Conv2d(in_channels = 64, out_channels =  64, kernel_size = 3)
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels =  64, kernel_size = 3)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(64 * 17 * 17, 64)
        self.fc2 = nn.Linear(64,1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        #print(x.size())
        
        x = x.view(-1, 64 * 17 * 17)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return  x
           
net = Net().cuda()
criterion =  nn.BCEWithLogitsLoss()
optimizer = optim.RMSprop(net.parameters(),lr=0.01)

In [0]:
def train_model(model, train_loader,val_loader, criterion, optimizer, num_epochs=1):
    since = time.time()
    iter = 0
    for epoch in range(2): 
      running_loss = 0.0
      for i,(inputs, labels) in enumerate(train_loader):
        inputs,labels = Variable(inputs.cuda()), Variable(labels.cuda())
        
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output.view(-1),labels.float())
        loss.backward()
        optimizer.step()   
            
      correct = 0
      total = 0    
      
      with torch.no_grad():
        for val_input, val_labels in val_loader:
          val_input = Variable(val_input.cuda())
          output = model(val_input)
          _, predicted = torch.max(output.data,1)
          total += val_labels.size(0)
          correct += (predicted.cpu() == val_labels.cpu()).sum()
        accuracy = 100 * correct / total
        print('epoch: {} val Loss {} val Accuracy {}'.format(epoch+1, loss.item(), accuracy))

        
    print('\n')
    time_elapsed = time.time() - since
    return model
  
model_ft = train_model(net,  train_loader,val_loader,  criterion, optimizer)